In [1]:
import pandas as pd
import numpy as np
import datetime
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv("D:\OneDrive - NITT\Custom_Download\Abstimmung_1433_3_monts.csv",
                 delimiter=';',encoding='ANSI')
shop_number = df[df['KST']!=0]['KST'].unique()

In [2]:
df.shape

(1595, 8)

In [3]:
len(shop_number)

27

In [4]:
def get_index(df,date,promised_amount,no_loopup_days=5):
    payment = df[['DATE','Haben']]
    offset = date + datetime.timedelta(days=no_loopup_days)
    
    small_df = payment[(payment['DATE']<= offset) & (payment['DATE']>= date)]
    small_df = small_df[~small_df['Haben'].isnull()]
    
    y = small_df[small_df['Haben']==promised_amount]
    indx = y.index[0] if len(y)>=1 else None
    return indx

In [5]:
pair_dict = dict()
for shop in sorted(shop_number):
    pair_list = []
    data = df[df['KST'].isin([shop]) | df['KST2'].isin([shop])]
    data['Soll'] = np.where(data['KST']==shop,data['BETRAG'],np.nan)
    data['Haben'] = np.where(data['KST2']==shop,data['BETRAG'],np.nan)

    data = data[['DATE','KST','KST2','Soll','Haben']]

    data['DATE'] = pd.to_datetime(data['DATE'],dayfirst=True,infer_datetime_format=True)

    data_clean = data[['DATE','Soll']][~data['Soll'].isnull()]
    for indx, row in data_clean.iterrows():
        indx_payment = get_index(data,row['DATE'],row['Soll'])
        Search_indx = sum(pair_list,())
        
        if row['Soll']<0:
            indx_payment = data_clean[(data_clean['DATE']== row['DATE']) & (data_clean['Soll']== abs(row['Soll']))].index[0]
            
            pair_list.append((indx_payment,indx)) 
        else:
            if indx_payment!=None:
                if indx_payment not in Search_indx:
                    pair_list.append((indx,indx_payment))
                    
    pair_dict[shop] = pair_list
                
indx_drop = sorted(sum(sum(pair_dict.values(),[]),()))

In [6]:
df_final = df.drop(indx_drop)

In [7]:
df.shape

(1595, 8)

In [8]:
len(df_final) + len(indx_drop)

1595

In [9]:
len(indx_drop)

1064

In [10]:
len(set(indx_drop))

1064

In [ ]:
from collections import Counter
d = Counter(indx_drop)

In [ ]:
for k,v in d.items():
    if v==2:
        print(k)

In [ ]:
x = sum(pair_dict.values(),[])

In [ ]:
for i,j in x:
    if i==516 or j==516:
        print(i,j)

In [ ]:
df.iloc[[442,504,516]]

In [ ]:
import pandas as pd
pd.set_option('display.max_rows', 1700)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
shop = 11
df[df['KST'].isin([shop]) | df['KST2'].isin([shop])]